## Case 6 - DRO opt DRJCC with ESS reserve but without revenue
## Case - SSA
## Case - robust
## Case - momentDRO
## Case - risk_neural

### Author: Junhyeok Kim

In [1]:
# Import the library

import os
import pandas as pd
import numpy as np
import sys
import time

import matplotlib.pyplot as plt
np.set_printoptions(formatter={'float': '{: 0.6f}'.format})

In [2]:
# Define the Variables
BASE_DIR = os.getcwd()
os.chdir("../")
path = os.getcwd() 
sys.path.append(path) # 폴더 한 단계 위에서 file import 하기 위해서 sys path 설정
sys.path.append(f"{path}/src")
sys.path.append(f"{path}/src/Data_Generation")

from rts_data import generate_wind, generate_gen_dict
from aggregator import aggregator
from gurobiMILP import gurobi_MILP, oos_test
from draw_fig import Single_Case_Plot
from different_approach import gurobi_simulation

In [3]:
### Parameters
DRO_param = {'eps_joint_cvar' : 0.05}
DRO_param['theta'] = 0.5

# Vector for Bonferroni approximation
rho_vectorC = np.linspace(0, 0.0025, 26)

# Number of individual runs (number of coupled datasets in the numerical study)

IR_max = 10


# Number of maximum sample size (N)

N_max = 200

# Number of maximum out of sample data for each individual run (N') for testing
# dataset
OOS_max = 1000

# Number of sample data in training dataset (N)

N = 100;


# Total number of data 

n_total_scen = IR_max * (N_max + OOS_max)

In [4]:
# Define the Parameters

# case_dict: Generate various scenario
# res_var: Define the WT and PV as variables [True] or parameters [False]
# case:
    # case 1 : w/o uncertainty
    # case 2 : w uncertainty with DRO
    #UNIT_TIME: 1 -> 1 hour
    
case_dict = {'case':6, 'UNIT_TIME': 1, 'bid_type':'deterministic', 'n_total_scen': n_total_scen, 'N_max': N_max, 'OOS_max':OOS_max,
             'IR_max': IR_max, 'N': N, 'divide_factor': 1}
case_dict['data_scenario'] = 'random'
case_dict['wt_bound'] = 0.6
case_dict['pv_bound'] = 0.2
case_dict['rand'] = 'uniform'

case_dict['date'] = '20220911'
nTimeslot = int(24 / case_dict['UNIT_TIME'])
char_ess = {'initSOC':0.5, 'termSOC':0.5, 'minSOC':0.2, 'maxSOC':0.8, 'efficiency':0.95}


model_dict = {'nVPP':1, 'path': path, 'N_PIECE': 10, 'nTimeslot': nTimeslot, 'output_flag': 0}

# Once, set te PV, WT, SMP uncertainties identically in each time (PV: 5%, WT: 10%, SMP: 10%)
uncertainty_dict = {'pv': np.ones(nTimeslot)*0.10 , 'wt': np.ones(nTimeslot)*0.10, 'smp':np.ones(nTimeslot)*0.10}


if case_dict['case']==2:
    
    model_dict['uncertainty'] = uncertainty_dict
    case_dict['bid_type'] = 'risky'
    
elif case_dict['case'] == 1:
    
    uncertainty_dict = {'pv': np.zeros(nTimeslot), 'wt': np.zeros(nTimeslot), 'smp':np.zeros(nTimeslot)}
    model_dict['uncertainty'] = uncertainty_dict
    

In [5]:
# Set Up VPP  #Unit: kW

vpp_list = []
for i in range(model_dict['nVPP']):
    nGen_dict = {'WT':4,'PV':2, 'ESS':1, 'DG':2}
    wt_list = [1800, 2000, 1700, 2000]
    pv_list = [1000, 850, 700, 700]
    ess_list = [400]
    capacity_list = [2000]
    dg_list = [800, 700, 900]
    max_list = [wt_list, pv_list, ess_list, capacity_list, dg_list]
    
    dg_dict_list = []
    for j in range(nGen_dict['DG']):
        dg_dict_list.append(generate_gen_dict(j,dg_list[j], model_dict))
    model_dict['dg_dict_list'] = dg_dict_list
    
    agg_dict = {'name': f'cvpp{i+1}', 'code': f'xds{i+1}', 'gen':nGen_dict}
    vpp_list.append(aggregator(agg_dict, char_ess, model_dict, case_dict))
    vpp_list[i].set_der_power(max_list)
    vpp_list[i].set_smp_data(case_dict['date'])
    vpp_list[i].set_profile(case_dict['N'],0)
    vpp_list[i].set_res_cov()
    
vpp_list[0].get_res_table()

Error
'aggregator' object has no attribute 'wt_uncert'
Aggregator set_res_table method
Uncertainty does not exist
Data Generated by historical data
Data Generated by historical data
Data Generated by historical data
Data Generated by historical data
Set_Res_Cov
Set_Res_Cov


,name,type,number,min_power,max_power,capacity
0,WT1_cvpp1,WT,1,0.0,1800,NaN
1,WT2_cvpp1,WT,2,0.0,2000,NaN
2,WT3_cvpp1,WT,3,0.0,1700,NaN
3,WT4_cvpp1,WT,4,0.0,2000,NaN
4,PV5_cvpp1,PV,5,0.0,1000,NaN
5,PV6_cvpp1,PV,6,0.0,850,NaN
6,ESS7_cvpp1,ESS,7,-400.0,400,2000.0
7,DG8_cvpp1,DG,8,80.0,800,NaN
8,DG9_cvpp1,DG,9,70.0,700,NaN


In [6]:
profile_std = np.zeros([vpp_list[0].nTimeslot,1])
for i in range(vpp_list[0].nRES):
    profile_std += vpp_list[0].res_list[i].profile_std
print(profile_std)

[[ 880.020014]
 [ 766.926185]
 [ 903.883019]
 [ 893.970902]
 [ 892.158747]
 [ 829.216351]
 [ 966.520599]
 [ 961.506935]
 [ 1089.881732]
 [ 1157.375466]
 [ 1233.509014]
 [ 859.021096]
 [ 486.688810]
 [ 507.856373]
 [ 825.898324]
 [ 769.855664]
 [ 759.417978]
 [ 641.439081]
 [ 720.130564]
 [ 609.270957]
 [ 635.420843]
 [ 509.411249]
 [ 292.782519]
 [ 375.601143]]


In [7]:
vpp_list[0].sum_res_profile_xi.std(axis=1)

array([ 394.468157,  371.630519,  442.340415,  443.751114,  433.400946,
        398.228236,  413.399763,  456.663485,  517.591453,  573.872045,
        517.603893,  371.907945,  209.443053,  200.882744,  379.301112,
        330.464118,  388.466970,  316.284136,  363.295582,  317.061445,
        307.777204,  255.669513,  137.320262,  182.034863])

In [8]:
profile_xi = np.zeros((24,100))
for i in range(vpp_list[0].nRES):
    profile_xi += vpp_list[0].res_list[i].profile_xi 
    
print(profile_xi.min(axis=1))

print(profile_xi.max(axis=1))

[-1067.087193 -1140.451218 -988.632136 -948.081693 -1203.633456
 -1000.218012 -915.615192 -1304.119030 -1157.167995 -1541.040241
 -1598.064946 -976.991109 -542.455277 -443.289773 -973.401200 -948.636721
 -982.808252 -836.537616 -831.187249 -796.234700 -662.844615 -490.762709
 -335.316622 -579.548445]
[ 1107.876620  816.945447  1146.456699  1021.893551  1010.905123
  995.654248  928.374118  1119.100973  1175.318976  1253.530815
  1087.915200  971.120531  629.747043  522.826921  853.414260  754.932556
  954.094731  735.404885  1042.424554  727.606999  731.858595  636.328299
  282.477298  417.236892]


In [9]:
# Gurobi Optimization Model
Wmax = vpp_list[0].wt_list[0].max_power 
Wmu = vpp_list[0].wt_list[0].profile_mu
#case_dict['theta'] = DRO_param['theta']* Wmu
#case_dict['theta'] = np.reshape(case_dict['theta'], -1)
case_dict['theta'] = [DRO_param['theta']] * nTimeslot
case_dict['eps'] = DRO_param['eps_joint_cvar']
case_dict['beta'] = 0.1
case_dict['alpha_max'] = 0.2
case_dict['GRID_PIECE'] = 100
case_dict['calc_rad'] = True
#case_dict['theta'] = np.array([0.05]*24)

mip_gap = 0.0001
feas_tol = 1e-4
time_limit = 600
case_dict['case'] = 6

# WDRCC Scenario Test

In [10]:
opt_bid_WDRCC = gurobi_MILP('opt bid_WDRCC', vpp_list[0], model_dict, case_dict)
sol, obj_dict, P_dict, U_dict, slack_dict = opt_bid_WDRCC.optimize(mip_gap, feas_tol)

Does not Cosidered alpha
Add Bid Constraint
Calculate the Radius
Calculated Theta:  0.07035359602113568
start set_dro_obj_constriants
start drjcc
iteration 0 of sum drjcc
iteration 5 of sum drjcc
iteration 10 of sum drjcc
iteration 15 of sum drjcc
iteration 20 of sum drjcc
finish max constraint of DRJCC
start set_base_objectives
end set_base_objectives
Optimal Solution:
Optimization Duration Time: 4.501232385635376


In [11]:
w1= opt_bid_WDRCC.P_dict['dg_ru']
w2= opt_bid_WDRCC.P_dict['RU_essdis']
w3= opt_bid_WDRCC.P_dict['RU_esschg']
w4 = opt_bid_WDRCC.m.objVal
print(w4)
print(w1[0]+w1[1]+w2+w3)
print(w1,w2,w3)

10105185.899028039
[[ 407.070081  372.377141  493.226912  440.765875  464.465242  380.711470
   429.679497  446.348652  552.843468  573.223027  404.897985  231.881016
   6.734260  0.000000  154.994241  179.446184  289.537766  238.986858
   230.000000  167.227223  212.655398  191.404443  41.617359  122.411925]]
[[ 240.000000  240.000000  213.226912  183.346001  207.045368  145.871721
   240.000000  240.000000  240.000000  240.000000  206.779002  80.000000
   0.000000  0.000000  84.994241  160.000000  204.493429  168.986858
   160.000000  160.000000  160.000000  140.808679  41.617359  122.411925]
 [ 167.070081  132.377141  280.000000  257.419874  257.419874  234.839748
   189.679497  206.348652  280.000000  280.000000  198.118984  151.881016
   6.734260  0.000000  70.000000  19.446184  85.044337  70.000000
   70.000000  7.227223  52.655398  50.595763  0.000000  0.000000]] [[ 0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000
   0.000000  32.843468  53.223027  0.000000  

In [12]:
lhs_array, rhs_array, check_wdrcc_array, ratio_wdrcc = oos_test(opt_bid_WDRCC, vpp_list[0], 1000,'WDRCC')
print(check_wdrcc_array.mean(axis=1))
print(ratio_wdrcc)

[ 0.972000  0.967000  0.954000  0.931000  0.964000  0.935000  0.946000
  0.964000  0.983000  0.980000  0.930000  0.941000  0.942000  0.955000
  0.931000  0.945000  0.952000  0.964000  0.948000  0.951000  0.940000
  0.980000  0.958000  0.941000]
0.9530833333333333


# SAA Scenario Test

In [13]:
case_dict['case'] = 'SAA'
opt_bid_SAA = gurobi_simulation('opt bid_SAA', vpp_list[0], model_dict, case_dict)
sol_SAA, obj_dict_SAA, P_dict_SAA, U_dict_SAA, slack_dict_SAA = opt_bid_SAA.optimize(mip_gap, feas_tol, time_limit) 

Does not Cosidered alpha
Add Bid Constraint
start set_saa_obj_constriants
end set_saa_obj_constriants
start SAA-cc
iteration 0 of sum SAA_jcc
iteration 5 of sum SAA_jcc
iteration 10 of sum SAA_jcc
iteration 15 of sum SAA_jcc
iteration 20 of sum SAA_jcc
finish max constraint of SAA_JCC
start set_base_objectives
Consider Case for DRO at base
end set_base_objectives
Optimal Solution:
Optimization Duration Time: 0.6667056083679199


In [14]:
s1= opt_bid_SAA.P_dict['dg_ru']
s2= opt_bid_SAA.P_dict['RU_essdis']
s3= opt_bid_SAA.P_dict['RU_esschg']
s4 = opt_bid_SAA.m.objVal

print(s4)
print(s1[0]+s1[1]+s2+s3)
print(s1,s2,s3)

10750166.851532556
[[ 288.469550  242.970235  411.242993  420.257557  385.460376  315.214358
   366.376780  248.418625  433.612426  441.130031  325.464739  191.946180
   0.000000  0.000000  65.322371  60.184946  248.736983  192.431756
   182.126426  141.330453  87.562598  127.067781  14.135562  59.298391]]
[[ 218.469550  172.970235  155.798504  185.702046  166.876643  96.630625
   209.209313  178.418625  240.000000  240.000000  115.464739  80.000000
   0.000000  0.000000  0.000000  60.184946  178.736983  160.000000
   160.000000  141.330453  87.562598  127.067781  14.135562  59.298391]
 [ 70.000000  70.000000  255.444489  234.555511  218.583734  218.583734
   157.167467  70.000000  193.612426  201.130031  210.000000  111.946180
   0.000000  0.000000  65.322371  0.000000  70.000000  32.431756
   22.126426  0.000000  0.000000  0.000000  0.000000  0.000000]] [[ 0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000
   0.000000  0.000000  0.000000  0.000000  0.000000  0.00000

In [15]:
oos_lhs_array, oos_rhs_array, oos_SAA_check_array, ratio_SAA = oos_test(opt_bid_SAA, vpp_list[0], 1000, 'SAA')

print(oos_SAA_check_array.mean(axis=1))
print(ratio_SAA)


[ 0.951000  0.934000  0.929000  0.929000  0.941000  0.925000  0.944000
  0.940000  0.965000  0.966000  0.905000  0.933000  0.949000  0.963000
  0.894000  0.898000  0.946000  0.952000  0.944000  0.940000  0.904000
  0.968000  0.950000  0.905000]
0.9364583333333333


# Robust Scenario Test

In [16]:
case_dict['case'] = 'robust'
opt_bid_robust = gurobi_simulation('opt bid_robust', vpp_list[0], model_dict, case_dict)
sol_robust, obj_dict_robust, P_dict_robust, U_dict_robust, slack_dict_robust = opt_bid_robust.optimize(mip_gap, feas_tol, time_limit)

Add Variable Robust
Does not Cosidered alpha
Add Bid Constraint
No need Robust Constraint
start set_base_objectives
Consider Case for robust at base
end set_base_objectives
Optimal Solution:
Optimization Duration Time: 0.02003955841064453


In [17]:
r1= opt_bid_robust.P_dict['dg_ru']
r2= opt_bid_robust.P_dict['RU_essdis']
r3= opt_bid_robust.P_dict['RU_esschg']
r4 = opt_bid_robust.m.objVal

print(r4)
print(r1[0]+r1[1]+r2+r3)
print(r1,r2,r3)



5602621.888438437
[[ 0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000
   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000
   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000
   0.000000  0.000000  0.000000]]
[[ 0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000
   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000
   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000
   0.000000  0.000000  0.000000]
 [ 0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000
   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000
   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000
   0.000000  0.000000  0.000000]] [[ 0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000
   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000
   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000
   0.000000  0.00

In [18]:
oos_lhs_array, oos_rhs_array, oos_robust_check_array, ratio_robust = oos_test(opt_bid_robust, vpp_list[0], 1000, 'robust')
print(oos_robust_check_array.mean(axis=1))
print(ratio_robust)



[ 0.800000  0.779000  0.710000  0.693000  0.692000  0.754000  0.823000
  0.846000  0.826000  0.803000  0.743000  0.839000  0.956000  0.963000
  0.868000  0.875000  0.837000  0.831000  0.823000  0.903000  0.838000
  0.895000  0.964000  0.848000]
0.8295416666666666


In [19]:
worst = np.zeros(24)
for i in range(5):
    for t in range(24):
        worst[t] += opt_bid_robust.res_list[i].worst_profile_xi[t]
print(worst)

[-1391.589647 -1355.746062 -1643.919818 -1566.815354 -1598.163345
 -1482.183548 -1630.077689 -1561.838754 -1762.756666 -1943.832010
 -2052.855982 -1382.560040 -724.364688 -786.095663 -1334.345633
 -1302.341579 -1278.964733 -1141.732110 -1247.833351 -1057.850127
 -1095.240245 -835.390268 -528.377687 -671.846528]


# Moment-DRO Scenario Test

In [20]:
from scipy.stats import norm
z_score = norm.ppf(1-0.05)
print(z_score)
profile_xi_std = np.zeros(24)
for t in range(24):
    for rr in range(5):
        profile_xi_std[t] += vpp_list[0].res_list[rr].profile_xi[t].std()
print(profile_xi_std * z_score)

1.6448536269514722
[ 1445.890942  1259.995154  1484.998587  1468.604171  1466.062015
  1362.296405  1588.215184  1574.596852  1746.186921  1818.218279
  1923.262640  1298.015819  683.161729  720.757233  1268.053272
  1209.960197  1225.727297  1052.036343  1182.822225  1000.524230
  1043.651148  836.258971  480.097793  616.111426]


In [21]:
case_dict['case'] = 'moment_DRO'
model_dict['gamma1'] = 0.1
model_dict['gamma2'] = 1.1
#case_dict['moment_eps'] = 'ki'
case_dict['moment_eps'] = 'z_score'

opt_bid_momentDRO = gurobi_simulation('opt bid_momentDRO', vpp_list[0], model_dict, case_dict)
sol_momentDRO, obj_dict_momentDRO, P_dict_momentDRO, U_dict_momentDRO, slack_dict_momentDRO = opt_bid_momentDRO.optimize(mip_gap, feas_tol, time_limit)

Does not Cosidered alpha
Add Bid Constraint
start set_momentDRO_obj_constriants
end set_momentDRO_obj_constriants
ki =  4.69041575982343
start moment_drjcc
iteration 0 of sum moment_drjcc
iteration 5 of sum moment_drjcc
iteration 10 of sum moment_drjcc
iteration 15 of sum moment_drjcc
iteration 20 of sum moment_drjcc
finish max constraint of moment_DRJCC
start set_base_objectives
Consider Case for momentDRO at base
end set_base_objectives
Optimal Solution:
Optimization Duration Time: 0.062001705169677734


In [22]:
m1= opt_bid_momentDRO.P_dict['dg_ru']
m2= opt_bid_momentDRO.P_dict['RU_essdis']
m3= opt_bid_momentDRO.P_dict['RU_esschg']
m4 = opt_bid_momentDRO.m.objVal

print(m4)
print(m1[0]+m1[1]+m2+m3)
print(m1,m2,m3)

9511072.219556203
[[ 326.322196  294.777874  444.202340  468.978061  399.209464  344.573645
   292.776899  362.668614  449.914137  486.069958  430.469167  267.497813
   26.305808  0.000000  216.725443  168.732343  318.112396  229.178233
   253.847402  199.059501  207.968378  167.779262  0.000000  102.849667]]
[[ 239.999999  224.777875  207.238831  192.761169  185.426355  134.573645
   152.776901  239.999999  239.999999  240.000000  239.999999  80.000001
   0.000002  0.000000  146.725444  160.000000  200.000000  160.000000
   183.847401  160.000000  160.000000  120.000000  0.000000  102.849666]
 [ 86.322197  69.999999  236.963509  276.216891  213.783109  210.000000
   139.999998  122.668615  209.914137  246.069958  190.469168  187.497813
   26.305806  0.000000  69.999999  8.732343  118.112396  69.178233
   70.000000  39.059501  47.968378  47.779263  0.000000  0.000000]] [[ 0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000
   0.000000  0.000000  0.000000  0.000000  0.0

In [23]:
oos_lhs_array, oos_rhs_array, oos_momentDRO_check_array, ratio_momentDRO = oos_test(opt_bid_momentDRO, vpp_list[0], 1000, 'momentDRO')
print(oos_momentDRO_check_array.mean(axis=1))
print(ratio_momentDRO)

[ 0.952000  0.944000  0.929000  0.932000  0.935000  0.923000  0.915000
  0.950000  0.961000  0.966000  0.930000  0.949000  0.951000  0.953000
  0.945000  0.931000  0.953000  0.940000  0.954000  0.956000  0.938000
  0.968000  0.930000  0.927000]
0.943


In [24]:
rhs = np.zeros(24)
lhs= np.zeros(24)
rhs2 = np.zeros(24)

rhs_slack1 = np.zeros(24)
rhs_slack2 = np.zeros(24)
rhs_slack3 = np.zeros(24)
rhs_2norm = np.zeros(24)

theta_0 = np.zeros(24)
theta_1 = np.zeros(24)
theta_2 = np.zeros(24)
lambda_o = np.zeros(24)
const = 1000
for i in range(24):
    theta_0[i] = opt_bid_momentDRO.theta_0[i].X
    theta_1[i] = opt_bid_momentDRO.theta_1[i].X
    theta_2[i] = opt_bid_momentDRO.theta_2[i].X
    lambda_o[i] = opt_bid_momentDRO.lambda_o[i].X

    rhs_slack1[i] = opt_bid_momentDRO.rhs_slack1[i].X
    rhs_slack2[i] = opt_bid_momentDRO.rhs_slack2[i].X
    rhs_slack3[i] = opt_bid_momentDRO.rhs_slack3[i].X
    rhs_2norm[i] = opt_bid_momentDRO.rhs_2norm[i].X
    lhs[i] = (theta_2[i] + lambda_o[i])/const
    rhs[i] =  np.sqrt((rhs_slack1[i])**2 + rhs_slack2[i]**2 + rhs_slack3[i]**2)
    rhs2[i] = rhs_2norm[i]
print("lhs")
print(lhs)
print()
print("rhs")
print(rhs)
print()
print("rhs2")
print(rhs2)
print()
print("rhs - rhs2")
print(rhs-rhs2)

lhs
[ 90.565408  84.844945  94.652843  94.736067  92.378774  84.807868
  101.165845  112.493751  121.411521  134.834644  110.419485  77.137861
  46.642390  44.852068  86.979936  77.491150  137.053769  119.961185
  138.298143  120.296627  116.664278  96.540727  51.432450  46.807527]

rhs
[ 90.565408  84.844945  94.652843  94.736067  92.378774  84.807868
  101.165845  112.493751  121.411521  134.834644  110.419485  77.137861
  46.642390  44.852068  86.979936  77.491150  137.053769  119.961185
  138.298143  120.296627  116.664278  96.540727  51.432450  46.807527]

rhs2
[ 90.565408  84.844945  94.652843  94.736067  92.378774  84.807868
  101.165845  112.493751  121.411521  134.834644  110.419485  77.137861
  46.642390  44.852068  86.979936  77.491150  137.053769  119.961185
  138.298143  120.296627  116.664278  96.540727  51.432450  46.807527]

rhs - rhs2
[ 0.000000 -0.000000  0.000000  0.000000  0.000000  0.000000  0.000000
  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.0

In [25]:
theta_1

array([ 62032.536296,  57760.594975,  66348.780069,  66456.396692,
        64611.668039,  58681.192103,  69158.704167,  77754.522698,
        85878.176176,  96818.365005,  79206.216298,  53101.584568,
        30261.352645,  29014.024187,  59319.534402,  51970.418697,
        89949.913319,  77230.068768,  89816.975815,  77456.007070,
        74987.239644,  61439.600582,  31976.830364,  29856.881962])

In [26]:
theta_2

array([ 59549.157675,  55964.668180,  61478.463954,  61507.879284,
        60072.952768,  55467.289009,  66586.510028,  73616.505730,
        78472.439441,  86425.469103,  70816.379036,  50587.089402,
        31511.725146,  30345.067856,  57320.189253,  51505.964168,
        92078.820587,  81346.161465,  93389.662351,  81568.626812,
        79170.668453,  65820.952567,  35444.035426,  31879.097631])

In [27]:
print(theta_0 + lambda_o)

[ 46471.987419  43949.629192  46782.252954  46764.869810  45828.004942
  42822.584270  52068.734387  56900.337016  59088.071153  63885.436392
  52414.832458  39269.991258  25874.690554  24978.613531  44932.559468
  41049.717869  74859.238861  67226.482390  76618.593216  67403.204795
  65514.537685  54902.859562  30091.142766  26543.189790]


In [28]:
print(theta_2 - lambda_o)
print(rhs_slack1)

print("const1")

print(theta_1)
print(rhs_slack2)
print("const2")

print(rhs_slack3)
print("const3")



[ 28532.907610  27084.391446  28304.085329  28279.691823  27767.131305
  26126.710480  32007.174614  34739.260280  35533.358237  38016.294682
  31213.273319  24036.317805  16381.060240  15838.067256  27660.442854
  25520.778556  47103.871835  42731.137522  48481.181751  42840.626428
  41677.059256  35101.177781  19455.621320  16950.668160]
[ 28.532908  27.084391  28.304085  28.279692  27.767131  26.126710
  32.007175  34.739260  35.533358  38.016295  31.213273  24.036318
  16.381060  15.838067  27.660443  25.520779  47.103872  42.731138
  48.481182  42.840626  41.677059  35.101178  19.455621  16.950668]
const1
[ 62032.536296  57760.594975  66348.780069  66456.396692  64611.668039
  58681.192103  69158.704167  77754.522698  85878.176176  96818.365005
  79206.216298  53101.584568  30261.352645  29014.024187  59319.534402
  51970.418697  89949.913319  77230.068768  89816.975815  77456.007070
  74987.239644  61439.600582  31976.830364  29856.881962]
[ 62.032536  57.760595  66.348780  66.45

In [29]:
case_dict['case'] = 'risk_neural'

opt_bid_neural = gurobi_simulation('opt bid_neural', vpp_list[0], model_dict, case_dict)
sol_neural, obj_dict_neural, P_dict_neural, U_dict_neural, slack_dict_neural = opt_bid_neural.optimize(mip_gap, feas_tol, time_limit)

Does not Cosidered alpha
Add Bid Constraint
start set_base_objectives
No Considered Any Case at base_Objectives
No obj_sum_without_bid before this
end set_base_objectives
Optimal Solution:
Optimization Duration Time: 0.020999908447265625


In [30]:
n1= opt_bid_neural.P_dict['dg_ru']
n2= opt_bid_neural.P_dict['RU_essdis']
n3= opt_bid_neural.P_dict['RU_esschg']
n4 = opt_bid_neural.m.objVal

print(n4)
print(n1[0]+m1[1]+m2+m3)
print(n1,n2,n3)

10861752.1015997
[[ 86.322197  69.999999  236.963509  276.216891  213.783109  210.000000
   139.999998  122.668615  209.914137  246.069958  190.469168  187.497813
   26.305806  0.000000  69.999999  8.732343  118.112396  69.178233
   70.000000  39.059501  47.968378  47.779263  0.000000  0.000000]]
[[ 0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000
   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000
   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000
   0.000000  0.000000  0.000000]
 [ 0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000
   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000
   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000
   0.000000  0.000000  0.000000]] [[ 0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000
   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000
   0.000000  0.000000  0.000000  0.000000  0.000000  0.0000

In [31]:
oos_lhs_array, oos_rhs_array, oos_neural_check_array, ratio_neural = oos_test(opt_bid_neural, vpp_list[0], 1000, 'neural')
print(oos_neural_check_array.mean(axis=1))
print(ratio_neural)

[ 0.800000  0.779000  0.710000  0.693000  0.692000  0.754000  0.823000
  0.846000  0.826000  0.803000  0.743000  0.839000  0.956000  0.963000
  0.868000  0.875000  0.837000  0.831000  0.823000  0.903000  0.838000
  0.895000  0.964000  0.848000]
0.8295416666666666


In [32]:
print("WDRCC :", w4)
print("SAA : ", s4)
print("robust : ", r4)
print("momentDRO : ", m4)
print("Risk_Neural :", n4)

WDRCC : 10105185.899028039
SAA :  10750166.851532556
robust :  5602621.888438437
momentDRO :  9511072.219556203
Risk_Neural : 10861752.1015997


In [33]:
print("Prob:")
print("WDRCC :", ratio_wdrcc)
print("SAA : ", ratio_SAA)
print("robust : ", ratio_robust)
print("momentDRO : ", ratio_momentDRO)
print("Risk_Neural :", ratio_neural)

Prob:
WDRCC : 0.9530833333333333
SAA :  0.9364583333333333
robust :  0.8295416666666666
momentDRO :  0.943
Risk_Neural : 0.8295416666666666


In [34]:
x1 = 1000
x2 = 1234
x3 =8947


z0 = np.sqrt(x1**2+x2**2 + x3**2)
z1 = np.sqrt((x1/100)**2+(x2/100)**2 + (x3/100)**2)
print(z0)


print(x1)

print(z0/z1)

9086.889731915977
1000
100.0


# Risk-Neural